In [1]:
import requests # Apache License 2.0
from requests.auth import HTTPBasicAuth

import uuid     # in python
import base64   # in python
import yaml     # MIT


from app.utils import get_data_offer, offer2et, create_poc_ContractRequest_body
from app.utils import str_edc_catalog

In [2]:
# --- variables ---
with open('consumer_cfg.yaml', 'r') as file:
    consumer_cfg = yaml.safe_load(file)

# - control plane -
url_edc_consumer_control_plane_base = consumer_cfg['consumer-edc-control-plane']['endpoint']
header_control_plane = consumer_cfg['consumer-edc-control-plane']['header'] # this contains secrets, so please use -at least- a secretsmanager instead

# - "identities" -
edc_provider_bpn = consumer_cfg['trusted-providers']['Facotry_Operator_A']['BPN']  # "{{EDCTX-10-1-BPN}}"
url_edc_provider_control_plane_base = consumer_cfg['trusted-providers']['Facotry_Operator_A']['endpoint-control-plane']

In [3]:
# -------------------------------------------------------------------------------------------------------------------

In [19]:
object_of_agreement = 'simple_test' # we 'magically' know this due to the push notification

In [ ]:
# see if there are some edrs which have beennegotiated for
agreement_body = {
    "@context": {
        "@vocab": "https://w3id.org/edc/v0.0.1/ns/"
    },
    "@type": "QuerySpec",
    "filterExpression": [
        {
            "operandLeft": "assetId",
            "operator": "=",
            "operandRight": object_of_agreement,
        }
    ]
}

res_catalog_agreement = requests.post(url=url_edc_consumer_control_plane_base + '/management/v2/edrs/request', headers=header_control_plane, json=agreement_body)
res_catalog_agreement

In [ ]:
res_offer, offer = get_data_offer(res_catalog_agreement.json())
print("Status Offer: "+ str(res_offer))

if res_offer == 0:
    res_et, et_dict = offer2et(offer, url_edc_consumer_control_plane_base, header_control_plane)
    if res_et == -2:
        print("Token Request Failed:" + str(et_dict))
    else:
        print("Status Endpoint and Token: " + str(res_et))
    

In [7]:
# <<< if the above says: no offer available, GOTO the chapter "Negotiate for Asset" below and return here once the contract is finalized

In [ ]:
# obtain data:
res_data_info = requests.get(url=et_dict['endpoint'], headers={'Authorization': et_dict['token']})
res_data_info

In [ ]:
res_data = requests.get(url=et_dict['endpoint'] + '/$value?extent=WithBlobValue', headers={'Authorization': et_dict['token']})
res_data

In [ ]:
val = res_data.json()[res_data_info.json()['submodelElements'][0]['idShort']]['value']
base64.b64decode(val).decode('utf-8')

### Negotiate for Asset

In [ ]:
# obtain all offers from the data provider using a catalog request:
catalog_request_body = {  
    "@context": {
        "@vocab": "https://w3id.org/edc/v0.0.1/ns/",
        "odrl":   "http://www.w3.org/ns/odrl/2/",
        "cx-taxo": "https://w3id.org/catenax/taxonomy#"
    },
    "@type": "CatalogRequest",
    "counterPartyId":      edc_provider_bpn,
    "counterPartyAddress": url_edc_provider_control_plane_base + "/api/v1/dsp",
    "protocol": "dataspace-protocol-http", 
    "querySpec": {
        "@type": "QuerySpec",
        "filterExpression": [
            {
                "operandLeft": "'http://purl.org/dc/terms/type'.'@id'",
                "operator": "=",
                "operandRight": "https://w3id.org/catenax/taxonomy#Submodel"  # <- here we say what we look for!
            }
        ]
    }
}

In [ ]:
# note: we query against our own EDC (the consumer EDC, who then will negotiate with the target EDC)
res_catalog = requests.post(url=url_edc_consumer_control_plane_base + '/management/v2/catalog/request', headers=header_control_plane, json=catalog_request_body)
print(str_edc_catalog(res_catalog))

In [10]:
# obtian offer and endpoint
for dcat_dataset in res_catalog.json()['dcat:dataset']:
    # look for the dataset with our id:
    if dcat_dataset['@id'] == object_of_agreement:
        asset_policy = dcat_dataset['odrl:hasPolicy']
        offer_id     = asset_policy['@id']

        # get negotiation endpoint: dct_endpointUrl
        dct_endpointUrl = None
        for distribution_method in dcat_dataset['dcat:distribution']:
            if distribution_method['dct:format']['@id'] == 'HttpData-PULL':
                dct_endpointUrl = distribution_method['dcat:accessService']['dct:endpointUrl']
                break
        # check if we actuall got the desired endpoint        
        if dct_endpointUrl is not None:
            break

In [ ]:
# create request body for the EDR negotiate
edr_negotiation_body = create_poc_ContractRequest_body(dct_endpointUrl, offer_id, edc_provider_bpn, object_of_agreement)
res_edr_negotiation  = requests.post(url=url_edc_consumer_control_plane_base + '/management/v2/edrs', headers=header_control_plane, json=edr_negotiation_body)
res_edr_negotiation		

In [ ]:
edr_negotation_id = res_edr_negotiation.json()['@id']
edr_negotation_id   # <- necessary to get the state 

In [ ]:
# get the negotiation state:
res_get_edr_negotiation_state = requests.get(url=url_edc_consumer_control_plane_base + '/management/v2/contractnegotiations/' + edr_negotation_id + '/state', headers=header_control_plane)
res_get_edr_negotiation_state

In [ ]:
res_get_edr_negotiation_state.json()    # <- this should have finalized